### Splitiing the data set to test and train

حواسم باشم رندم و شافل و انجام ندم

In [9]:
import pandas as pd
import random
train_frac = 0.8

rating_path = '../data/ml-latest-small/ratings.csv'
main_rating_pd = pd.read_csv(rating_path)

num_user = np.unique(rating_pd.values[:, 0]).shape[0]
num_movie = np.unique(rating_pd.values[:, 1]).shape[0]

df_shuffled = rating_pd.sample(frac=1).reset_index(drop=True)
# df_shuffled.head()

#train_set = df_shuffled.iloc[:int(train_frac * len(df_shuffled))]
train_set.to_csv(path_or_buf='../data/train_set.csv', index=False)
#test_set = df_shuffled.iloc[int(train_frac * len(df_shuffled)):]
test_set.to_csv(path_or_buf='../data/test_set.csv', index=False)

In [10]:
train_path = '../data/train_set.csv'
len(pd.read_csv(train_path))

80668

In [11]:
import numpy as np
import pandas as pd

In [12]:
latent_dim = 100
sigma = 0.5
sigma_p = 0.5
sigma_q = 0.5
mu = 0
rating_path = '../data/ml-latest-small/ratings.csv'
train_path = '../data/train_set.csv'

In [55]:
rating_pd = pd.read_csv(train_path)

rating_pd.head()
len(rating_pd['movieId'].unique())

8973

In [35]:
r_matrix = np.zeros((num_user, num_movie))
p_matrix = np.random.normal(mu, sigma_p, (num_user, latent_dim))
q_matrix = np.random.normal(mu, sigma_q, (num_movie, latent_dim))

print('shape p matrix: {}'.format(p_matrix.shape))
print('shape q matrix: {}'.format(q_matrix.shape))

movie_dic = {}
idx = 0
for movie_id in np.unique(rating_pd.values[:, 1]):
    movie_dic[movie_id] = idx
    idx += 1

for index, row in rating_pd.iterrows():
    r_matrix[int(row['userId'] - 1), movie_dic[row['movieId']]] = row['rating']

shape p matrix: (610, 100)
shape q matrix: (9724, 100)


In [57]:
rating_path = '../data/ml-latest-small/ratings.csv'
main_rating_pd = pd.read_csv(rating_path)


In [64]:
movie_IDD = set(main_rating_pd['movieId'].unique().tolist())
train_ID = set(rating_pd['movieId'].unique().tolist())
not_in_training_ID = movie_IDD - train_ID

In [65]:
import operator
sorted_dic = sorted(movie_dic.items(), key=operator.itemgetter(1))

with open('../doc/movie_indexes.csv', 'w') as f:
    f.write('matrix_index,movie_id\n')
    for movie_id, index in sorted_dic:
        f.write('{},{}\n'.format(index, int(movie_id)))

In [66]:
index = 8973
with open('../doc/movie_indexes.csv', 'a') as f:
    for movie_id in not_in_training_ID:
        f.write('{},{}\n'.format(index, int(movie_id)))
        index += 1

In [37]:
def MSE (r, p, q):
    p_qt_matrix = np.dot(p, q.T)
    result = r - p_qt_matrix
    I = np.zeros_like(r)
    I[r != 0] = 0.5
    result = np.power(result, 2)
    result = np.multiply(I, result)
    return np.sum(result)
    
def L2_loss(sig1, sig2, matrix):
    result = np.power(matrix, 2)
    result = np.sum(result)
    constant = sig1 / float(sig2 * 2)
    return constant * result    

In [38]:
error = MSE(r_matrix, p_matrix, q_matrix) + L2_loss(sigma, sigma_q, q_matrix) + L2_loss(sigma, sigma_p, p_matrix)

In [39]:
def p_q_derivative (r, p, q, sigma, sigma_p, sigma_q):
    p_qt_matrix = np.dot(p, q.T)
    residual = r - p_qt_matrix
    I = np.zeros_like(r)
    I[r != 0] = 1
    residual = np.multiply(I, residual)
    p_derivative = -1.0 * np.dot(residual, q) + sigma / sigma_p * p
    q_derivative = -1.0 * np.dot(residual.T, p) + sigma / sigma_q * q
    return p_derivative, q_derivative

In [44]:
big_lr = 0.001
small_lr = 0.0001

cnt = 0 
while (error >= 238):
    
    if error > 239:
        learning_rate = big_lr
    else:
        learning_rate = small_lr
    
    learning_rate = 0.001   
    p_derivative, q_derivative = p_q_derivative (r_matrix, p_matrix, q_matrix, sigma, sigma_p, sigma_q)
    p_matrix = p_matrix - learning_rate * p_derivative 
    q_matrix = q_matrix - learning_rate * q_derivative 
    error = MSE(r_matrix, p_matrix, q_matrix)
    cnt += 1
    if cnt % 100 == 0:
        print(error)
print(cnt)        

239.59623557524776
239.30000260400948
239.01107869438172
238.729196173908
238.45410282115412
238.1855601415257
671


In [45]:
p_matrix.shape

(610, 100)

In [49]:
import pandas as pd 
pd.DataFrame(p_matrix).to_csv("../output/p_matrix_A2_final.csv")
pd.DataFrame(q_matrix).to_csv("../output/q_matrix_A2_final.csv")